# Create a Cohort then Export the Data to S3

## Import Libraries and Set Cohort ID

In [1]:
import komodo
from komodo.client import Client

# Initialize Komodo Client
client = Client()
cohort_definition_id = "fltr_def_NRKYCATAYYCAFIKS"  # replace this cohort definition ID

## Run Your Cohort

In [2]:
# import the CohortCreate model from the Definitions API/business domain
from komodo.definitions.models.cohorts.cohort_create import CohortCreate

# create the JSON payload
cohort_payload = {
    "name": "Cohort",
    "definition": {
        "cohort_definition": {
            "filters": [{
                "name": "filter_dfn", 
                "filter_definition": {"filter_definition_id": cohort_definition_id},
                "time_filter": {  # optional
                    "ranges": [
                        ["2024-01-01", "2024-01-31"]
                    ]
                }
            }],
            "entities": ["patient"],
            "source_filter": {
                "version": "release", 
                "include_rejected_claims": False
            },
        },
        "output_format": {
            "count_entities": False,
            "entities_to_count": [],
            "outputs": [{
                "output_format": "snowflake-table",
                "output_type": "plaid-providers",  # based on your subscription
            }],
        },
    },
    "definition_schema_version": "1.0.0",
}

# create the CohortCreate instance with the JSON payload
cohort_create = CohortCreate.from_dict(cohort_payload)

In [3]:
# create the Komodo client
from komodo.client import Client
client = Client()

# call the create_cohort operation
cohort_create_response = client.definitions.create_cohort(cohort_create)

# save the ID of the cohort
cohort_id = cohort_create_response.id

# print the cohort ID
cohort_id

# store the cohort ID as an environment variable that can be used across cookbook files
from dotenv import load_dotenv, set_key
set_key('.env', 'cohort_id', cohort_id)

from time import sleep
 
cohort_response = client.definitions.get_cohort(cohort_id)
 
while cohort_response.cohort_run.finished_at is None and cohort_response.cohort_run.error_message is None:
    cohort_response = client.definitions.get_cohort(cohort_id)
    print(f"Cohort status is {cohort_response.cohort_run.status.value}")
    if cohort_response.cohort_run.status == "FINISHED":
        break
    sleep(5)

Cohort status is QUEUED
Cohort status is RUNNING
Cohort status is RUNNING
Cohort status is RUNNING
Cohort status is RUNNING
Cohort status is RUNNING
Cohort status is RUNNING
Cohort status is FINISHED


In [4]:
try:
    get_cohort_response = client.definitions.get_cohort(cohort_id)
except Exception as e:
    print("Exception when calling DefinitionsApi->get_cohort: %s\n" % e)


In [5]:
# create the Komodo client
from komodo.client import Client

client = Client()

# call the get_cohort operation
cohort_response = client.definitions.get_cohort(cohort_id=cohort_id)

# retrieve the dataset ID of the cohort output
cohort_snowflake_dataset_id = cohort_response.cohort_run.output.outputs[0].dataset_id

# call the get_dataset operation
cohort_snowflake_dataset2 = client.data_catalog.get_dataset(dataset_id=cohort_snowflake_dataset_id)

# retrieve the fully qualified name of the dataset table
cohort_snowflake_dataset_table2 = cohort_snowflake_dataset2.manifestations[0].fully_qualified_name

# print the fully qualified name of the dataset table
print(f"Dataset table: {cohort_snowflake_dataset_table2}")

Dataset table: COHORTS.PROD.COHORT_RUN_CHT_LVAEFROCIOSLOIBL_0_PLAID_PROVIDERS


## Convert Snowflake to Pandas 

In [6]:
# import pandas and the get_snowflake_connection function
import pandas as pd
from komodo.snowflake import get_snowflake_connection

# call the get_snowflake_connection and declare a Snowflake cursor
sf_connection = get_snowflake_connection()
sf_cursor = sf_connection.cursor()

# set the role of the cursor to CUSTOMER_ROLE to read and write tables
sf_cursor.execute("USE ROLE CUSTOMER_ROLE")

# retrieve the first 1,000 rows of the cohort data
cohort_data2 = pd.read_sql(f"select * from {cohort_snowflake_dataset_table2} LIMIT 1000", sf_connection)

# preview the first 5 rows of the current Snowflake table/Pandas DataFrame
cohort_data2.head()

/home/dragon/workspaces/current/workspace/.venv/lib/python3.11/site-packages/snowflake/connector/options.py:108: UserWarning: You have an incompatible version of 'pyarrow' installed (19.0.1), please install a version that adheres to: 'pyarrow<19.0.0; extra == "pandas"'
  warn_incompatible_dep(
DEBUG:komodo_connector.connection_creators.snowflake.connect:REST API object was created: f82bd78b-1a87-4b6c-a121-dd977d140a9d.snowflakecomputing.com:443
/tmp/ipykernel_87127/3802850886.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  cohort_data2 = pd.read_sql(f"select * from {cohort_snowflake_dataset_table2} LIMIT 1000", sf_connection)


,NPI,PROVIDER_TYPE,FIRST_NAME,LAST_NAME,ORGANIZATION_NAME,HCO_PRIMARY_NPI,PRIMARY_SPECIALTY,SECONDARY_SPECIALTY,PROVIDER_PHONE_NUMBER,PROVIDER_ADDRESS,PROVIDER_CITY,PROVIDER_STATE,PROVIDER_ZIP
0,1437713633,INDIVIDUAL,RENAE,BRIDGES,None,1184709057,Nurse Practitioner,Adult Health,3218419025,"1335 SLIGH BLVD, STE 3",ORLANDO,FL,32806
1,1437897899,ORGANIZATION,None,None,ACTIVE INFUSIONS INC,None,Clinic/Center,Infusion Therapy,2402004464,14502 GREENVIEW DR STE 360,LAUREL,MD,20708
2,1598417412,ORGANIZATION,None,None,GRMC INC,None,General Acute Care Hospital,None,3015534000,251 N 4TH ST,OAKLAND,MD,21550
3,1407886401,INDIVIDUAL,NOORUZSABHA,AHMED,None,1235125717,Internal Medicine,Infectious Disease,0022064328,2130 POINT BLVD STE 900,ELGIN,IL,60123
4,1407887003,ORGANIZATION,None,None,SELF REGIONAL HEALTHCARE,None,General Acute Care Hospital,None,8647254111,1325 SPRING ST,GREENWOOD,SC,29646


In [7]:
# import the current date and time
from datetime import datetime

now = datetime.now()

from komodo.dataset import upload_dataset_to_maplab

# set the name of the dataset to be uploaded to the Komodo platform
# add the current date and time to the end of the dataset name to make it more distinct
cohort_upload_datetime = now.strftime("%Y%m%d_%H%M%S")
cohort_upload_dataset_name = "COHORT_DATASET" + cohort_upload_datetime

# call the upload_dataset_to_maplab function
cohort_upload_dataset = upload_dataset_to_maplab(cohort_data2, cohort_upload_dataset_name)

# save the ID of the dataset
dataset_id = cohort_upload_dataset.id

# print the dataset ID
dataset_id

# store the dataset ID as an environment variable that can be used in subsequent cookbook files
from dotenv import load_dotenv, set_key

set_key(".env", "dataset_id", dataset_id)

DEBUG:komodo_connector.connection_creators.snowflake.connect:REST API object was created: f82bd78b-1a87-4b6c-a121-dd977d140a9d.snowflakecomputing.com:443


(True, 'dataset_id', '64690376-1b89-4911-a0a6-5bd1a56dfb73')

In [8]:
# use this cell if you want to use the dataset ID that is stored in an environment variable

import os
from dotenv import load_dotenv
load_dotenv()

try:
    dataset_id = os.environ["dataset_id"]  # retrieve the dataset_id from the "3-retrieve-cohort-data.ipynb" cookbook file
except KeyError:
    print("Please set the variable `dataset_id` with value of your dataset id.")  # throw an error if no dataset ID can be retrieved

## Export Your Dataset

In [9]:
import pprint

# create the Komodo client
from komodo.client import Client
client = Client()

# retrieve list of Share requests
share_ids = client.connections.list_shares()
pprint.pprint(share_ids.shares)

[ShareMetadata(account_id='f82bd78b-1a87-4b6c-a121-dd977d140a9d', connection_string='arn:aws:iam::851851261022:role/PDI-s3-export', created_by='3009a19d-57af-47c1-a141-eec85b519c9f', created_time='2025-03-19 15:35:27.612861+00:00', customer_region='us-west-2', customer_s3_export_path='s3://kh-studio-test-pdi-s3-export/sandbox-maplab-enterprise', database_name=None, listing_name=None, output_file_format=<OutputFileFormatEnum.CSV: 'CSV'>, region='us-west-2', share_id='9bb1938d-3718-4e3c-9345-be89c73337de', share_name='sandboxmaplabenterprise_s3_external', share_type=<DataFormatEnum.S3_EXTERNAL: 'S3_EXTERNAL'>, updated_by='3009a19d-57af-47c1-a141-eec85b519c9f', updated_time='2025-03-19 15:35:27.612861+00:00', user_id='3009a19d-57af-47c1-a141-eec85b519c9f')]


In [10]:
share_id = '9bb1938d-3718-4e3c-9345-be89c73337de' # str | 

try:
    get_share_response = client.connections.get_share(share_id)
except Exception as e:
    print("Exception when calling ConnectionsApi->get_share: %s\n" % e)


In [11]:
# retrieve the required IAM role policies
s3_details = client.connections.get_shares_aws_details()

s3_iam_policy = s3_details.iam_policy_json
print(s3_iam_policy)

s3_iam_trust_relationship = s3_details.iam_trust_relationship_json
print(s3_iam_trust_relationship)

{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Action": ["s3:GetObject", "s3:GetObjectTagging", "s3:GetObjectVersion", "s3:GetObjectVersionTagging", "s3:ListBucket"], "Resource": ["arn:aws:s3:::pdi-export-bb376550-8b58-4a65-b03c-bb0e8e564a02-prod-uw2", "arn:aws:s3:::pdi-export-bb376550-8b58-4a65-b03c-bb0e8e564a02-prod-uw2/sds_dataset_shares/*"]}, {"Effect": "Allow", "Action": ["s3:DeleteObject", "s3:GetObject", "s3:GetObjectAcl", "s3:ListBucket", "s3:PutObject", "s3:PutObjectAcl"], "Resource": ["arn:aws:s3:::{customer_s3_bucket}/{customer_s3_path}/*", "arn:aws:s3:::{customer_s3_bucket}"]}, {"Effect": "Allow", "Action": ["kms:Decrypt", "kms:GenerateDataKey"], "Resource": ["arn:aws:kms:us-west-2:904233131998:key/mrk-54689dae89204a768488fe09f9968257", "arn:aws:kms:{customer_region}:{customer_aws_account_id}:key/{customer_bucket_key_id}"]}]}
{"Version": "2012-10-17", "Statement": [{"Effect": "Allow", "Principal": {"AWS": ["arn:aws:iam::904233131998:role/customer_account_role"

In [12]:
from komodo.data_deliveries.models.dataset_shares.create_dataset_share_request import CreateDatasetShareRequest

dataset_export_request = CreateDatasetShareRequest(dataset_ids=[dataset_id])
dataset_export_response = client.data_deliveries.create_dataset_share(share_id=share_id, create_dataset_share_request=dataset_export_request)
dataset_export_id = dataset_export_response.dataset_shares[0].dataset_share_id

dataset_export_id

ServiceException: (502)
Reason: Bad Gateway
HTTP response headers: HTTPHeaderDict({'Date': 'Fri, 18 Apr 2025 15:22:01 GMT', 'Content-Type': 'application/json', 'Content-Length': '112', 'Connection': 'keep-alive', 'x-request-id': '0787087fdeae46ba9cadf93d5e848ebd', 'Access-Control-Allow-Origin': '*'})
HTTP response body: {"message":"Error submitting the scan for dataset: 64690376-1b89-4911-a0a6-5bd1a56dfb73: Internal Server Error"}
